train classifier 

In [112]:
import csv 
import pandas as pd 

In [113]:
df = pd.read_csv('all_data.csv')

In [114]:
data = df.headline
target = df.is_clickbait

In [115]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(\
    data, target, test_size=0.20, random_state=42)

In [116]:
X_train.head(5)

6029         How Far Would You Make It In The Hunger Games
2937     If Matthew Gray Gubler's Tweets Were Motivatio...
16186           European Shares Mixed After a Jump in Asia
13762    53 Important Questions The Internet Has About ...
21264    13,000 people participate in Bristol running e...
Name: headline, dtype: object

In [117]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

In [118]:
#what would happen with IDF=true?
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [119]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', penalty='l2'\
    ,alpha=1e-3, random_state=42,max_iter=5, tol=None).fit(X_train_tf, y_train)


In [120]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect',CountVectorizer()),
                     ('tfidf',TfidfTransformer()),
                    ('clf',SGDClassifier()),
                    ])

In [121]:
text_clf.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [122]:
import numpy as np 
predicted = text_clf.predict(X_test)

In [123]:
np.mean(predicted==y_test)

0.97046875

In [124]:
from sklearn import metrics
metrics.confusion_matrix(y_test,predicted)

array([[3086,   64],
       [ 125, 3125]])

In [126]:
file = pd.read_csv('finished_csv/aljazeera.com.csv')


In [ ]:
file